In [ ]:
# Colab setup code block
try:
    import google.colab
    print("Running on Google Colab. Setting up environment...")
    
    import os
    if not os.path.exists('CMAI-E91-Students'):
        !git clone https://github.com/algolab-quantique/CMAI-E91-Students.git

    os.chdir('CMAI-E91-Students/assignment')
    !pip install -r ../requirements.txt -q
    print("Environment setup complete!")
except ImportError:
    print("Not running on Google Colab. Skipping setup.")


# Défi de programmation - E91 avec |Φ-⟩

Lors de l'atelier, nous avons implémenté le protocole E91 en utilisant l'**état singulet** |Ψ-⟩ = (|01⟩ - |10⟩)/√2.

Dans ce devoir, vous allez adapter l'E91 pour utiliser l'**état de Bell |Φ-⟩** = (|00⟩ - |11⟩)/√2.

**Différence clé :**
- |Ψ-⟩ donne des résultats **anti-corrélés** (Alice=0 → Bob=1)  
- |Φ-⟩ donne des résultats **corrélés** (Alice=0 → Bob=0)

## Ce que vous allez apprendre :

1. **Tout état de Bell fonctionne** pour l'E91, mais il faut utiliser la bonne formule !  
2. Les états **corrélés vs anti-corrélés** influencent l'extraction de la clé  
3. **La formule CHSH change** selon l'état de Bell utilisé  
4. **Approche expérimentale** : essayez toutes les possibilités et trouvez celle qui donne $2\sqrt{2}$

**À FAIRE :** Adaptez le protocole E91 pour utiliser l'état de Bell |Φ-⟩ et déchiffrez les messages secrets.  
- Créez le circuit |Φ-⟩ (astuce : ajoutez UNE porte à |Φ+⟩)  
- Trouvez la formule CHSH correcte (essayez les 4 positions du signe moins !)  
- Complétez le protocole E91 et déchiffrez les messages


### Configuration

In [ ]:
# ============================================================
# SETUP - All imports and constants (no external notebook imports!)
# ============================================================

import random
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import sys
sys.path.append('utils')
import encryption_algorithms as enc

# Simulator setup
aer_simulator = AerSimulator()
MANUAL_SIMULATOR_SEED_COUNTER = 42

# Use a DIFFERENT seed for this assignment!
GLOBAL_SEED = 42  # Different from workshop (91)
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# E91 Protocol Constants
ALICE_BASES = ['0', '45', '90']
BOB_BASES = ['45', '90', '135']
ALICE_CHSH_BASES = ['0', '90']
BOB_CHSH_BASES = ['45', '135']
CHSH_BASIS_PAIRS = [(a, b) for a in ALICE_CHSH_BASES for b in BOB_CHSH_BASES]
BELL_INEQUALITY_THRESHOLD = 2.0

print(" Setup complete - fully self-contained!")
print(f"\nAlice's bases: {ALICE_BASES}")
print(f"Bob's bases: {BOB_BASES}")
print(f"CHSH basis pairs: {CHSH_BASIS_PAIRS}")


## Tâche 1 : Créer l'état de Bell |Φ-⟩ (15 points)

Complétez la fonction pour créer |Φ-⟩ = (|00⟩ - |11⟩)/√2

**Astuce :** Commencez par |Φ+⟩ et ajoutez UNE porte pour créer le signe moins !

```
|Φ+⟩: H → CX       gives (|00⟩ + |11⟩)/√2
|Φ-⟩: H → ? → CX   gives (|00⟩ - |11⟩)/√2
```

**Rappel :** La porte Z ajoute une phase π : |1⟩ → -|1⟩


In [ ]:
def create_bell_pair_phi_minus() -> QuantumCircuit:
    """
    TODO: Create the Bell state |Φ-⟩ = (|00⟩ - |11⟩)/√2
    
    Hint: Start with |Φ+⟩ (H + CX) and add ONE gate to create the minus sign!
    """
    qc = QuantumCircuit(2)
    
    # TODO: Add your gates here
    # qc.h(0)     # Hadamard on qubit 0
    # qc.?        # What gate creates the minus sign?
    # qc.cx(0,1)  # CNOT
    
    return qc

In [ ]:
# Test your circuit
phi_minus = create_bell_pair_phi_minus()
print("Your circuit for |Φ-⟩:")
print(phi_minus.draw())

### Vérification de l'état |Φ-⟩

Assurez-vous que vous avez bien créé le bon état de Bell.  
Astuce : $1/\sqrt{2}$ est identique à $\sqrt{2}/2$ 😄


In [ ]:
from qiskit.quantum_info import Statevector
Statevector(phi_minus).draw('latex')

## Fonctions auxiliaires (fournies)

Ce sont les mêmes fonctions que celles de l'atelier. Exécutez cette cellule !


In [ ]:
# ============================================================
# PART 1: CHSH BELL INEQUALITY - Core Functions
# ============================================================

def run_circuit(circ: QuantumCircuit, shots=1) -> dict:
    """Run a quantum circuit on the simulator."""
    global MANUAL_SIMULATOR_SEED_COUNTER
    global aer_simulator
    
    current_run_seed = MANUAL_SIMULATOR_SEED_COUNTER
    MANUAL_SIMULATOR_SEED_COUNTER += 1
    
    circ = transpile(circ, aer_simulator)
    result = aer_simulator.run(circ, shots=shots, seed_simulator=current_run_seed).result()
    return result.get_counts(circ)


def apply_basis_transformation(circuit: QuantumCircuit, qubit_index: int, basis: str) -> QuantumCircuit:
    """
    Apply rotation to measure in a specific basis (angle in degrees).
    
    Basis angles:
    - '0'   : Z basis (computational)
    - '45'  : Rotate by -π/4
    - '90'  : X basis (Hadamard)
    - '135' : Rotate by -3π/4
    """
    transformed = circuit.copy()
    
    if basis == '0':
        pass  # No rotation for Z basis
    elif basis == '90':
        transformed.h(qubit_index)
    elif basis == '45':
        transformed.ry(-np.pi/4, qubit_index)
    elif basis == '135':
        transformed.ry(-3*np.pi/4, qubit_index)
    else:
        raise ValueError(f"Unknown basis: {basis}")
    
    return transformed


def measure_bell_pair(circuit: QuantumCircuit, alice_basis: str, bob_basis: str) -> str:
    """Measure a Bell pair with specified bases, return result like '00', '01', etc."""
    meas_qc = circuit.copy()
    meas_qc = apply_basis_transformation(meas_qc, 0, alice_basis)
    meas_qc = apply_basis_transformation(meas_qc, 1, bob_basis)
    meas_qc.measure_all()
    
    counts = run_circuit(meas_qc, shots=1)
    return list(counts.keys())[0]


def organize_measurements_by_basis(results, alice_bases, bob_bases):
    """Group measurement results by basis pair for correlation calculation."""
    unique_alice = list(set(alice_bases))
    unique_bob = list(set(bob_bases))
    
    counts = {}
    for a in unique_alice:
        for b in unique_bob:
            counts[(a, b)] = {'00': 0, '01': 0, '10': 0, '11': 0}
    
    for i, result in enumerate(results):
        a_base = alice_bases[i]
        b_base = bob_bases[i]
        if result in counts[(a_base, b_base)]:
            counts[(a_base, b_base)][result] += 1
    
    return counts


def calculate_correlations(measurements):
    """
    Calculate E(a,b) correlation for each basis pair.
    
    Formula: E(a,b) = (N_same - N_different) / N_total
                    = (N_00 + N_11 - N_01 - N_10) / N_total
    """
    correlations = {}
    
    for basis_pair, results in measurements.items():
        total = sum(results.values())
        
        if total > 0:
            E = (results['00'] + results['11'] - results['01'] - results['10']) / total
            correlations[basis_pair] = E
        else:
            correlations[basis_pair] = 0
    
    return correlations
    

print(" Part 1: CHSH Bell Inequality functions loaded!")

## Vérification : Testez votre circuit |Φ-⟩

Exécutez ceci pour vérifier si votre circuit est correct. Vous devriez voir environ 50 % de |00⟩ et 50 % de |11⟩.


In [ ]:
# Test the |Φ-⟩ Bell state
phi_minus = create_bell_pair_phi_minus()

# Verify with measurements
test_qc = phi_minus.copy()
test_qc.measure_all()
counts = run_circuit(test_qc, shots=1000)

print(f"\nMeasurement results: {counts}")
print("\n Expected: ~50% |00⟩ and ~50% |11⟩ (never |01⟩ or |10⟩)")
print("   This confirms |Φ-⟩ is CORRELATED : ")
print("   --> Alice and Bob get SAME results: Alice=0 → Bob=0, Alice=1 → Bob=1")

print("\n Note: The minus sign (phase) doesn't affect Z-basis measurements,")
print("   but it DOES affect measurements in rotated bases (used for CHSH)!")

## Tâche 2 : Trouver la bonne formule CHSH

La formule CHSH contient UN signe moins. Pour |Φ-⟩, il se déplace vers une position différente !

**Formule de l'atelier (pour |Ψ-⟩) :**
$$S = E(a,b) - E(a,b') + E(a',b) + E(a',b')$$

**Votre tâche :** Essayez les 4 positions pour le signe moins et trouvez celle qui donne |S| ≈ 2√2 ≈ 2,83


**D’abord, nous générons des données de test pour |Φ⁻⟩ :**

In [ ]:
# Generate test data for |Φ-⟩

print("Generating 500 measurements for CHSH test...\n")
test_num_pairs = 500

# Generate random bases
test_alice_chsh_bases = [random.choice(ALICE_CHSH_BASES) for _ in range(test_num_pairs)]
test_bob_chsh_bases = [random.choice(BOB_CHSH_BASES) for _ in range(test_num_pairs)]

# Generate Bell pairs and Measure them
results = []
for a, b in zip(test_alice_chsh_bases, test_bob_chsh_bases):
    qc = create_bell_pair_phi_minus()
    results.append(measure_bell_pair(qc, a, b))


# step 4: Organize measurements by basis to compute correkations:
measurements = organize_measurements_by_basis(results, test_alice_chsh_bases, test_bob_chsh_bases)
correlations = calculate_correlations(measurements)

# Print correlations
print("Individual correlations E(a,b):")
for pair, E in sorted(correlations.items()):
    print(f"  E{pair} = {E:.4f}")

# Step 5: TODO
# Based on the correlations, compute and find the correct CHSH formulat (in next cell)

In [ ]:
# ═══════════════════════════════════════════════════════════
# TODO: Try all 4 minus sign positions!
# ═══════════════════════════════════════════════════════════

a1, a2 = '0', '90'   # Alice's CHSH bases
b1, b2 = '45', '135' # Bob's CHSH bases

E_ab = correlations[(a1, b1)]              # E(0°, 45°)
E_ab_prime = correlations[(a1, b2)]        # E(0°, 135°)
E_a_prime_b = correlations[(a2, b1)]       # E(90°, 45°)
E_a_prime_b_prime = correlations[(a2, b2)] # E(90°, 135°)

print("="*60)
print("Testing all 4 CHSH formula variations:")
print("="*60)

# TODO: Calculate S for each position of the minus sign

# Position 1: minus at first term
S_minus_p1 = 0  # TODO: -E_ab + E_ab_prime + E_a_prime_b + E_a_prime_b_prime
print(f"\n1. S = -E(a,b) + E(a,b') + E(a',b) + E(a',b')")
print(f"   |S| = {abs(S_minus_p1):.4f}")

# Position 2: minus at second term (original workshop formula)
S_minus_p2 = 0  # TODO: E_ab - E_ab_prime + E_a_prime_b + E_a_prime_b_prime
print(f"\n2. S = +E(a,b) - E(a,b') + E(a',b) + E(a',b')  [workshop formula]")
print(f"   |S| = {abs(S_minus_p2):.4f}")

# Position 3: minus at third term
S_minus_p3 = 0  # TODO: E_ab + E_ab_prime - E_a_prime_b + E_a_prime_b_prime
print(f"\n3. S = +E(a,b) + E(a,b') - E(a',b) + E(a',b')")
print(f"   |S| = {abs(S_minus_p3):.4f}")

# Position 4: minus at fourth term
S_minus_p4 = 0  # TODO: E_ab + E_ab_prime + E_a_prime_b - E_a_prime_b_prime
print(f"\n4. S = +E(a,b) + E(a,b') + E(a',b) - E(a',b')")
print(f"   |S| = {abs(S_minus_p4):.4f}")

print("\n" + "="*60)
print(f"Target: 2√2 ≈ {2*np.sqrt(2):.4f}")
print("The formula with |S| ≈ 2.83 is the CORRECT one!")
print("="*60)

### Votre réponse : Quelle formule CHSH est correcte ?

D'après vos résultats ci-dessus, indiquez la formule CHSH correcte pour |Φ-⟩ :


In [ ]:
def calculate_chsh_value_phi_minus(correlations, alice_bases=ALICE_CHSH_BASES, bob_bases=BOB_CHSH_BASES):
    """
    TODO: Implement the correct CHSH formula for |Φ-⟩
    
    Based on your experiment above, which position gives |S| ≈ 2√2?
    """
    a1, a2 = alice_bases  # '0', '90'
    b1, b2 = bob_bases    # '45', '135'
    
    # TODO: Write the correct formula here
    # S = ???
    S = 0  # Replace with the correct formula!
    
    
    return abs(S)


# Test your formula
chsh = calculate_chsh_value_phi_minus(correlations)
print(f"Your CHSH value: {chsh:.4f}")
print(f"Expected: ≈ {2*np.sqrt(2):.4f}")

## Fonctions auxiliaires du protocole E91

Ces fonctions orchestrent l’ensemble du protocole de distribution de clé E91.


In [ ]:
# ============================================================
# PART 2: E91 PROTOCOL - Helper Functions
# ============================================================

def generate_random_bases(length, options):
    """Generate a list of random measurement bases."""
    return [random.choice(options) for _ in range(length)]


def create_list_bell_pairs(num_pairs):
    """Create a list of |Φ-⟩ Bell pairs."""
    return [create_bell_pair_phi_minus() for _ in range(num_pairs)]


def measure_all_pairs(bell_pairs, alice_bases, bob_bases):
    """Measure all Bell pairs with the specified bases."""
    results = []
    for qc, a_base, b_base in zip(bell_pairs, alice_bases, bob_bases):
        result = measure_bell_pair(qc, a_base, b_base)
        results.append(result)
    return results


def extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases):
    """
    Sift measurement results into key generation and Bell test data.
    
    Rules:
    - Same basis (45,45) or (90,90) → Key generation
    - CHSH basis pairs → Bell test
    - Other combinations → Discard
    """
    key_results = []
    chsh_results = []
    chsh_alice_bases = []
    chsh_bob_bases = []
    
    for result, a_base, b_base in zip(results, alice_bases, bob_bases):
        if a_base == b_base:  # Same basis → Key
            key_results.append(result)
        elif (a_base, b_base) in CHSH_BASIS_PAIRS:  # CHSH bases → Security test
            chsh_results.append(result)
            chsh_alice_bases.append(a_base)
            chsh_bob_bases.append(b_base)
        # else: discard
    
    return {
        'key_results': key_results,
        'chsh_results': chsh_results,
        'chsh_alice_bases': chsh_alice_bases,
        'chsh_bob_bases': chsh_bob_bases,
    }


def check_for_eavesdropping(chsh_results, chsh_alice_bases, chsh_bob_bases):
    """Run CHSH security test on Bell test data."""
    bell_results = organize_measurements_by_basis(chsh_results, chsh_alice_bases, chsh_bob_bases)
    correlations = calculate_correlations(bell_results)
    chsh_value = calculate_chsh_value_phi_minus(correlations)
    
    return {
        'chsh_value': chsh_value,
        'is_secure': chsh_value > BELL_INEQUALITY_THRESHOLD
    }


print(" Part 2: E91 Protocol functions loaded!")


## Tâche 3 : Compléter le protocole E91 et déchiffrer les messages 
Maintenant, mettez tout cela ensemble ! Complétez la fonction `run_e91_protocol`.

**Rappel :**
- Utilisez votre état de Bell |Φ-⟩
- Utilisez votre formule CHSH correcte
- |Φ-⟩ est **corrélé** → Bob utilise les bits directement (pas d'inversion !)

In [ ]:
# ============================================================
# COMPLETE E91 PROTOCOL with |Φ-⟩
# ============================================================

# Reset seed for reproducibility
MANUAL_SIMULATOR_SEED_COUNTER = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

def run_e91_protocol(num_pairs=2000):
    """
    Run E91 protocol with |Φ-⟩ Bell state.
    
    TODO: Complete this function using:
    - Your create_bell_pair_phi_minus() function
    - Your calculate_chsh_value_phi_minus() function
    - Correct key extraction for CORRELATED state
    """
    print("="*60)
    print("E91 PROTOCOL with |Φ-⟩ Bell State")
    print("="*60)
    
    # Step 1: Generate Bell pairs
    print(f"\n1. Generating {num_pairs} |Φ-⟩ Bell pairs...")
    bell_pairs = create_list_bell_pairs(num_pairs)
    
    # Step 2: Generate random bases
    print("\n2. Alice and Bob choose random bases...")
    alice_bases = generate_random_bases(num_pairs, ALICE_BASES)
    bob_bases = generate_random_bases(num_pairs, BOB_BASES)
    
    # Step 3: Measure all pairs
    print("\n3. Measuring all pairs...")
    results = measure_all_pairs(bell_pairs, alice_bases, bob_bases)
    
    # Step 4: Sift results
    print("\n4. Sifting results...")
    data = extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases)
    print(f"   Key generation pairs: {len(data['key_results'])}")
    print(f"   CHSH test pairs: {len(data['chsh_results'])}")
    
    # Step 5: Security check
    print("\n5. Running CHSH security test...")
    security = check_for_eavesdropping(
        data['chsh_results'],
        data['chsh_alice_bases'],
        data['chsh_bob_bases']
    )
    print(f"\n   CHSH Value: {security['chsh_value']:.4f}")
    print(f"   Classical limit: 2.0")
    print(f"   Quantum limit: 2√2 ≈ 2.83")
    
    if not security['is_secure']:
        print("\n SECURITY CHECK FAILED!")
        print("   Possible eavesdropping detected.")
        return None
    
    print("\n SECURITY CHECK PASSED!")
    
    # Step 6: Extract key
    print("\n6. Extracting shared key...")
    
    # TODO: Key extraction for |Φ-⟩
    # Remember: |Φ-⟩ is CORRELATED → Bob uses bits DIRECTLY (no flip!)
    # Qiskit format: result = "BA" where B=qubit1, A=qubit0
    
    alice_key = ''.join([r[1] for r in data['key_results']])  # Alice = qubit 0
    bob_key = None    ## r[0] = Bob (qubit 1) - TODO: flip or not?

    
    if alice_key == bob_key:
        print("\n Keys match perfectly!")
    else:
        mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
        print(f"\n {mismatches} mismatches in {len(alice_key)} bits")
    
    print(f"   Key length: {len(alice_key)} bits")
    print(f"   Key (first 50): {alice_key[:50]}...")
    
    return alice_key


print(" E91 Protocol function loaded!")

### Exécuter le protocole E91


In [ ]:
key = run_e91_protocol(num_pairs=2000)

### Déchiffrement des messages secrets

Si votre protocole a été exécuté correctement, vous devriez pouvoir déchiffrer les messages !


In [ ]:
def decrypt_and_print_messages(key: str, filename: str = "encrypted_messages.txt"):
    """
    Read encrypted messages from a file, decrypt them using XOR with the given key, and print.

    Args:
        key (str): The key to use for XOR decryption.
        filename (str): The file to read encrypted messages from.
    """
    print("\nDecrypting all messages from", filename)
    with open(filename, "r") as f:
        for line in f:
            if ": " in line:
                msg_id, encrypted = line.split(": ", 1)
                decrypted = enc.decrypt_xor_repeating_key(encrypted.strip(), key)
                print(f"{msg_id}: {decrypted}")


In [ ]:
# For your challenge: must generate the correct key using the E91 protocol to decrypt the messages. Without the key, decryption is not feasible.
# change the path to the encrypted file as needed
path_to_encrypted_file = R"assignment_encrypted_messages.txt"
decrypt_and_print_messages(key, filename=path_to_encrypted_file)


## Résumé : Ce que vous avez appris

Complétez ce tableau en fonction de votre travail :

| Aspect             | Atelier ($\vert \Psi^- \rangle$)           | Ce Devoir ($\vert \Phi^- \rangle$) |
|-------------------|-------------------------------------------|------------------------------------|
| Circuit            | H → CX → X → Z                             | H → ??? → CX                       |
| Corrélation        | Anti-corrélée                              | ???                                |
| Extraction de clé  | Bob inverse les bits                        | ???                                |
| Formule CHSH       | $S = +E(a,b) - E(a,b') + E(a',b) + E(a',b')$ | $S = ???$                          |

**Aperçu clé :** N'importe quel état de Bell peut être utilisé pour l'E91, mais vous devez :  
1. Savoir s'il est **corrélé** ou **anti-corrélé** (pour l'extraction de la clé)  
2. Trouver la **bonne formule CHSH** (celle qui donne $|S| \approx 2\sqrt{2}$)
